In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
df = pd.read_pickle('df_pikabu_spam_posts.pd')

In [3]:
df = df.sample(frac=1)

In [4]:
words_set = set()

for row in df.itertuples():
    for word in row.title:
        words_set.add(word)
    for word in row.text:
        words_set.add(word)

print(f"Всего слов: {len(words_set)}")

Всего слов: 59073


In [5]:
# используем словарь для хранения кол-ва вхождений каждого слова
words_counter = {w: 0 for w in words_set}

for row in df.itertuples():
    for word in row.title:
        words_counter[word] += 1
    for word in row.text:
        words_counter[word] += 1

# преобразуем словарь в список и отсортируем его
words_list = list(words_counter.items())
words_list.sort(key=(lambda x: x[1]), reverse=True)

In [6]:
# оставим только 5к слов
words_list = words_list[:5000]

# кол-во вхождений нам уже не нужно
words_list = [k[0] for k in words_list]

# для быстрого создания OHE полезно будет заранее пронумеровать каждое слово
# чтобы не ждать выполнения операции получения позиции в списке
words_ohe_positions = {words_list[i]: i for i in range(len(words_list))}

In [7]:
# списки под заголовки и тексты, которые тоже будут закодированы списками
titles = []
texts = []

# перебираем все строки
for row in df.itertuples():
    # сначала создаём шаблок с одними нулями
    title_ohe = [0] * len(words_list)
    for word in row.title:
        try:
            # если слово из заголовка присутствует в нашем словаре, увеличиваем счётчик на соответствующем месте
            title_ohe[words_ohe_positions[word]] += 1
        except:
            # если слово отсутствует, словарь выкинет исключение - в таком случае просто продолжаем цикл
            continue
    # делаем то же самое и для текста поста
    text_ohe = [0] * len(words_list)
    for word in row.text:
        try:
            text_ohe[words_ohe_positions[word]] += 1
        except:
            continue
    # добавляем получившуюся кодировку в списки заголовков и текстов
    titles.append(title_ohe)
    texts.append(text_ohe)

# для работы с Keras информацию лучше держать в Numpy
titles = np.array(titles)
texts = np.array(texts)

In [8]:
y = np.array(df['bad'])

In [9]:
# чтобы не заниматься копипастом 3 раза, сделаем функцию,
# которая разделяет массив в нужной пропорции
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


titles_train, titles_val, titles_test = train_val_test_split(titles)
texts_train, texts_val, texts_test = train_val_test_split(texts)
y_train, y_val, y_test = train_val_test_split(y)

In [ ]:
#Задание 1

In [17]:
title_input = tf.keras.layers.Input(shape=(5000,))
text_input = tf.keras.layers.Input(shape=(5000,))

title_dense_1 = tf.keras.layers.Dense(500, activation='relu')(title_input)
text_dense_1 = tf.keras.layers.Dense(500, activation='relu')(text_input)

title_bn_1 = tf.keras.layers.BatchNormalization()(title_dense_1)
text_bn_1 = tf.keras.layers.BatchNormalization()(text_dense_1)

text_dense_2 = tf.keras.layers.Dense(500, activation='relu')(text_bn_1)

text_bn_2 = tf.keras.layers.BatchNormalization()(text_dense_2)

add = tf.keras.layers.Add()([title_bn_1, text_bn_2])

main_dense_1 = tf.keras.layers.Dense(300, activation='relu')(add)
main_bn_1 = tf.keras.layers.BatchNormalization()(main_dense_1)
drp1 = tf.keras.layers.Dropout(0.8)(main_bn_1)
main_dense_2 = tf.keras.layers.Dense(100, activation='relu')(drp1)
main_bn_2 = tf.keras.layers.BatchNormalization()(main_dense_2)
drp2 = tf.keras.layers.Dropout(0.8)(main_bn_2)

output = tf.keras.layers.Dense(1, activation='sigmoid')(drp2)

model = tf.keras.Model(inputs=[title_input, text_input], outputs=output)

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 5000)]       0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 500)          2500500     input_4[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 5000)]       0                                            
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 500)          2000        dense_7[0][0]                    
____________________________________________________________________________________________

In [ ]:
#Задание 2

In [18]:
accuracy = tf.keras.metrics.binary_accuracy
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
auc = tf.keras.metrics.AUC()

def f1_metrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec) / (prec + rec + 1e-7))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[accuracy,
                       precision,
                       recall,
                       f1_metrics,
                       auc])

In [19]:
import os

os.mkdir('logs')

In [20]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/first', 
                                             histogram_freq=1) # данный параметр нужен, чтобы мониторить гистограммы весов

In [21]:
model.fit([titles_train, texts_train], y_train,  # данные на вход указываем в списке в нужном порядке
          validation_data=([titles_val, texts_val], y_val),
          batch_size=256,
          epochs=10,
          callbacks=[tb_callback])

Epoch 1/10
21/21 [==============================] - 9s 212ms/step - loss: 1.2646 - binary_accuracy: 0.5176 - precision: 0.2691 - recall: 0.5600 - f1_metrics: 0.3629 - auc: 0.5420 - val_loss: 0.5844 - val_binary_accuracy: 0.7690 - val_precision: 0.5556 - val_recall: 0.1136 - val_f1_metrics: 0.1824 - val_auc: 0.6664
Epoch 2/10
21/21 [==============================] - 2s 80ms/step - loss: 0.9088 - binary_accuracy: 0.5951 - precision: 0.3226 - recall: 0.6833 - f1_metrics: 0.4289 - auc: 0.6840 - val_loss: 0.5289 - val_binary_accuracy: 0.8317 - val_precision: 0.9634 - val_recall: 0.2992 - val_f1_metrics: 0.4528 - val_auc: 0.8293
Epoch 3/10
21/21 [==============================] - 2s 81ms/step - loss: 0.5812 - binary_accuracy: 0.7424 - precision: 0.4747 - recall: 0.8066 - f1_metrics: 0.5935 - auc: 0.8383 - val_loss: 0.4422 - val_binary_accuracy: 0.8881 - val_precision: 0.9728 - val_recall: 0.5417 - val_f1_metrics: 0.6812 - val_auc: 0.9128
Epoch 4/10
21/21 [==============================] - 2s